In [2]:
# pytorch libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# for visualizing the results
import numpy as np
import matplotlib.pyplot as plt

# for reading input data
import pandas as pd

# for parsing the FEN of chess positions
import re

In [3]:
def fen_to_bit_vector(fen):
    # piece placement - lowercase for black pieces, uppercase for white pieces. numbers represent consequtive spaces. / represents a new row 
    # active color - whose turn it is, either 'w' or 'b'
    # castling rights - which castling moves are still legal K or k for kingside and Q or q for queenside, '-' if no legal castling moves for either player
    # en passant - if the last move was a pawn moving up two squares, this is the space behind the square for the purposes of en passant
    # halfmove clock - number of moves without a pawn move or piece capture, after 50 of which the game is a draw
    # fullmove number - number of full turns starting at 1, increments after black's move

    # Example FEN of starting position
    # rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1
    
    parts = re.split(" ", fen)
    piece_placement = re.split("/", parts[0])
    active_color = parts[1]
    castling_rights = parts[2]
    en_passant = parts[3]
    halfmove_clock = int(parts[4])
    fullmove_clock = int(parts[5])

    bit_vector = np.zeros((13, 8, 8), dtype=np.uint8)
    
    # piece to layer structure taken from reference [1]
    piece_to_layer = {
        'R': 1,
        'N': 2,
        'B': 3,
        'Q': 4,
        'K': 5,
        'P': 6,
        'p': 7,
        'k': 8,
        'q': 9,
        'b': 10,
        'n': 11,
        'r': 12
    }
    
    castling = {
        'K': (7,7),
        'Q': (7,0),
        'k': (0,7),
        'q': (0,0),
    }

    for r, row in enumerate(piece_placement):
        c = 0
        for piece in row:
            if piece in piece_to_layer:
                bit_vector[piece_to_layer[piece], r, c] = 1
                c += 1
            else:
                c += int(piece)
    
    if en_passant != '-':
        bit_vector[0, ord(en_passant[0]) - ord('a'), int(en_passant[1]) - 1] = 1
    
    if castling_rights != '-':
        for char in castling_rights:
            bit_vector[0, castling[char][0], castling[char][1]] = 1
    
    if active_color == 'w':
        bit_vector[0, 7, 4] = 1
    else:
        bit_vector[0, 0, 4] = 1

    if halfmove_clock > 0:
        c = 7
        while halfmove_clock > 0:
            bit_vector[0, 3, c] = halfmove_clock%2
            halfmove_clock = halfmove_clock // 2
            c -= 1
            if c < 0:
                break

    if fullmove_clock > 0:
        c = 7
        while fullmove_clock > 0:
            bit_vector[0, 4, c] = fullmove_clock%2
            fullmove_clock = fullmove_clock // 2
            c -= 1
            if c < 0:
                break

    return bit_vector

In [4]:
def fen_to_board_vector(fen):
    """
    We need to have a smaller representation of the board for the network to train on.
    """
    # piece placement - lowercase for black pieces, uppercase for white pieces. numbers represent consequtive spaces. / represents a new row 
    # active color - whose turn it is, either 'w' or 'b'
    # castling rights - which castling moves are still legal K or k for kingside and Q or q for queenside, '-' if no legal castling moves for either player
    # en passant - if the last move was a pawn moving up two squares, this is the space behind the square for the purposes of en passant
    # halfmove clock - number of moves without a pawn move or piece capture, after 50 of which the game is a draw
    # fullmove number - number of full turns starting at 1, increments after black's move

    # Example FEN of starting position
    # rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1
    
    parts = re.split(" ", fen)
    piece_placement = re.split("/", parts[0])
    active_color = parts[1]
    castling_rights = parts[2]
    en_passant = parts[3]
    halfmove_clock = int(parts[4])
    fullmove_clock = int(parts[5])

    board_vector = np.zeros( (64), dtype=np.float32) # 
    
    # piece to layer structure taken from reference [1]
    piece_to_value = {
        'R': 0.5,
        'N': 0.3,
        'B': 0.35,
        'Q': 0.9,
        'K': 1,
        'P': 0.1,
        'p': -0.1,
        'k': -1,
        'q': -0.9,
        'b': -0.35,
        'n': -0.3,
        'r': -0.5
    }
    
    castling = {
        'K': (7,7),
        'Q': (7,0),
        'k': (0,7),
        'q': (0,0),
    }

    for r, row in enumerate(piece_placement):
        c = 0
        for piece in row:
            if piece in piece_to_value:
                board_vector[r*8 + c] = piece_to_value[piece]
                c += 1
            else:
                c += int(piece)
    
    return board_vector

In [5]:
fen = "rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1"
board = fen_to_board_vector(fen)
print(board)



[-0.5  -0.3  -0.35 -0.9  -1.   -0.35 -0.3  -0.5  -0.1  -0.1  -0.1  -0.1
 -0.1  -0.1  -0.1  -0.1   0.    0.    0.    0.    0.    0.    0.    0.
  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.1   0.1   0.1   0.1   0.1   0.1   0.1   0.1   0.5   0.3   0.35  0.9
  1.    0.35  0.3   0.5 ]


In [19]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(64, 16, bias=False)
        self.fc2 = nn.Linear(16, 1)


    def forward(self, x):
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [20]:
# ChessDataset code and eval_to_int code taken from reference [1]
class ChessDataset(Dataset):
    def __init__(self, data_frame):
        self.fens = torch.from_numpy(np.array([*map(fen_to_board_vector, data_frame["FEN"])], dtype=np.float32))
        self.evals = torch.Tensor([[x] for x in data_frame["Evaluation"]])
        self._len = len(self.evals)
        
    def __len__(self):
        return self._len
    
    def __getitem__(self, index):
        return self.fens[index], self.evals[index]


def eval_to_int(evaluation):
    try:
        res = int(evaluation)
    except ValueError:
        res = 10000 if evaluation[1] == '+' else -10000
    return res / 100


In [22]:
def AdamW_main():
    MAX_DATA = 500000
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Using device {}".format(device))

    print("Preparing Training Data...")
    train_data = pd.read_csv("../data/chessData.csv")
    train_data = train_data[:MAX_DATA]
    train_data["Evaluation"] = train_data["Evaluation"].map(eval_to_int)
    trainset = ChessDataset(train_data)
    
    print("Preparing Test Data...")
    test_data = pd.read_csv("../data/tactic_evals.csv")
    test_data = test_data[:MAX_DATA]
    test_data["Evaluation"] = test_data["Evaluation"].map(eval_to_int)
    testset = ChessDataset(test_data)

    batch_size = 64

    print("Converting to pytorch Dataset...")

    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=6)

    testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=6)


    net = Net().to(device)
    criterion = nn.MSELoss()
    optimizer = optim.AdamW(net.parameters())


    for epoch in range(10):  # loop over the dataset multiple times

        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()

            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if i % 2000 == 1999:    # print every 2000 mini-batches
                # denominator for loss should represent the number of positions evaluated 
                # independent of the batch size
                print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / (2000*len(labels))))
                running_loss = 0.0

    print('Finished Training')

    PATH = './chesst_tiny_no_bias.pth'
    torch.save(net.state_dict(), PATH)

    print('Evaluating model')

    count = 0
    total_loss = 0
    # since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        for data in testloader:
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            #print("Correct eval: {}, Predicted eval: {}, loss: {}".format(labels, outputs, loss))
            
            # count should represent the number of positions evaluated 
            # independent of the batch size
            count += len(labels)
            total_loss += loss
            if count % 10000 == 0:
                print('Average error of the model on the {} tactics positions is {}'.format(count, loss/count))
    #print('Average error of the model on the {} tactics positions is {}'.format(count, loss/count))


In [23]:
AdamW_main()

Using device cuda
Preparing Training Data...
Preparing Test Data...
Converting to pytorch Dataset...
[1,  2000] loss: 2.312
[1,  4000] loss: 2.289
[1,  6000] loss: 2.286
[2,  2000] loss: 2.249
[2,  4000] loss: 2.215
[2,  6000] loss: 2.221
[3,  2000] loss: 2.193
[3,  4000] loss: 2.251
[3,  6000] loss: 2.066
[4,  2000] loss: 2.118
[4,  4000] loss: 2.105
[4,  6000] loss: 2.148
[5,  2000] loss: 2.057
[5,  4000] loss: 2.124
[5,  6000] loss: 2.147
[6,  2000] loss: 2.066
[6,  4000] loss: 2.062
[6,  6000] loss: 2.102
[7,  2000] loss: 1.955
[7,  4000] loss: 2.120
[7,  6000] loss: 2.100
[8,  2000] loss: 2.111
[8,  4000] loss: 2.041
[8,  6000] loss: 2.010
[9,  2000] loss: 2.017
[9,  4000] loss: 2.072
[9,  6000] loss: 2.070
[10,  2000] loss: 2.030
[10,  4000] loss: 2.032
[10,  6000] loss: 1.978
Finished Training
Evaluating model
Average error of the model on the 40000 tactics positions is 0.061318784952163696
Average error of the model on the 80000 tactics positions is 0.014967724680900574
Average

chess.pth:
Evaluating model
Average error of the model on the 40000 tactics positions is 0.0609552226960659
Average error of the model on the 80000 tactics positions is 0.016548501327633858
Average error of the model on the 100000 tactics positions is 0.03276893123984337


In [10]:
%pwd

'/home/mtrang/Documents/rl/Chess-Challenge/devel/notebooks'

In [25]:
import numpy as np
from scipy.interpolate import BSpline

# Assuming you have a dataset with 64 variables and a single output
# You can represent your dataset as a NumPy array, where each row is a data point
# and the last column contains the output values.

# Generate a sample dataset for demonstration purposes
num_data_points = 100
num_variables = 64

# Generate random input data

train_data = pd.read_csv("../data/chessData.csv")

MAX_DATA = 50000
train_data = train_data[:MAX_DATA]
train_data["FEN"] = train_data["FEN"].map(fen_to_board_vector)
train_data["Evaluation"] = train_data["Evaluation"].map(eval_to_int)
input_data = train_data["FEN"]

# Generate random output data
output_data = train_data["Evaluation"]


In [34]:
input_data[0][0]

-0.5

In [37]:

# Sort the data points based on the input variable of interest (e.g., the first variable)
sorted_indices = np.arange(len(input_data), dtype=np.int64)
sorted_input_data = input_data[sorted_indices]
sorted_output_data = output_data[sorted_indices]

# Define the degree of the spline (eighth degree)
degree = 8

# Create a B-spline object with the specified degree
k = degree + 1
t = np.linspace(0, 1, len(sorted_input_data))
spl = BSpline(t, sorted_output_data, k, extrapolate=True)

# Generate some test input values for interpolation
test_input = np.random.rand(10, num_variables)

# Interpolate the output values for the test input using the spline
interpolated_output = spl(t)

# Print the interpolated output
print("Interpolated Output:")
print(interpolated_output)


Interpolated Output:
[-259782.6107742   -93262.75236078  -29228.78977574 ...   26761.61759621
   86180.69226667  241982.53372402]


In [ ]:
import pygad
import numpy

"""
Given the following function:
    y = f(w1:w6) = w1x1 + w2x2 + w3x3 + w4x4 + w5x5 + 6wx6
    where (x1,x2,x3,x4,x5,x6)=(4,-2,3.5,5,-11,-4.7) and y=44
What are the best values for the 6 weights (w1 to w6)? We are going to use the genetic algorithm to optimize this function.
"""

function_inputs = [4,-2,3.5,5,-11,-4.7] # Function inputs.
desired_output = 44 # Function output.

def fitness_func(ga_instance, solution, solution_idx):
    # Calculating the fitness value of each solution in the current population.
    # The fitness function calulates the sum of products between each input and its corresponding weight.
    output = numpy.sum(solution*function_inputs)
    fitness = 1.0 / numpy.abs(output - desired_output)
    return fitness

fitness_function = fitness_func

num_generations = 100 # Number of generations.
num_parents_mating = 7 # Number of solutions to be selected as parents in the mating pool.

# To prepare the initial population, there are 2 ways:
# 1) Prepare it yourself and pass it to the initial_population parameter. This way is useful when the user wants to start the genetic algorithm with a custom initial population.
# 2) Assign valid integer values to the sol_per_pop and num_genes parameters. If the initial_population parameter exists, then the sol_per_pop and num_genes parameters are useless.
sol_per_pop = 50 # Number of solutions in the population.
num_genes = len(function_inputs)

last_fitness = 0
def callback_generation(ga_instance):
    global last_fitness
    print("Generation = {generation}".format(generation=ga_instance.generations_completed))
    print("Fitness    = {fitness}".format(fitness=ga_instance.best_solution()[1]))
    print("Change     = {change}".format(change=ga_instance.best_solution()[1] - last_fitness))
    last_fitness = ga_instance.best_solution()[1]

# Creating an instance of the GA class inside the ga module. Some parameters are initialized within the constructor.
ga_instance = pygad.GA(num_generations=num_generations,
                       num_parents_mating=num_parents_mating, 
                       fitness_func=fitness_function,
                       sol_per_pop=sol_per_pop, 
                       num_genes=num_genes,
                       on_generation=callback_generation)

# Running the GA to optimize the parameters of the function.
ga_instance.run()

# After the generations complete, some plots are showed that summarize the how the outputs/fitenss values evolve over generations.
ga_instance.plot_fitness()

# Returning the details of the best solution.
solution, solution_fitness, solution_idx = ga_instance.best_solution()
print("Parameters of the best solution : {solution}".format(solution=solution))
print("Fitness value of the best solution = {solution_fitness}".format(solution_fitness=solution_fitness))
print("Index of the best solution : {solution_idx}".format(solution_idx=solution_idx))

prediction = numpy.sum(numpy.array(function_inputs)*solution)
print("Predicted output based on the best solution : {prediction}".format(prediction=prediction))

if ga_instance.best_solution_generation != -1:
    print("Best fitness value reached after {best_solution_generation} generations.".format(best_solution_generation=ga_instance.best_solution_generation))

# Saving the GA instance.
filename = 'genetic' # The filename to which the instance is saved. The name is without extension.
ga_instance.save(filename=filename)

# Loading the saved GA instance.
loaded_ga_instance = pygad.load(filename=filename)
loaded_ga_instance.plot_fitness()